<a href="https://colab.research.google.com/github/chengpeip/colab_design_for_graduate/blob/main/orginal_levit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import timm
import math

# 设备设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 超参数
BATCH_SIZE = 32
EPOCHS = 20
WARMUP_EPOCHS = 5
BASE_LR = 0.0005
LR = BASE_LR * BATCH_SIZE / 10
WEIGHT_DECAY = 0.025
VAL_RATIO = 0.2

# 数据准备
dataset = EEGDataset('/content')

# 划分训练集和验证集
val_size = int(len(dataset) * VAL_RATIO)
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 模型初始化并移动到 GPU
model = timm.create_model('levit_128.fb_dist_in1k', pretrained=True, num_classes=2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# 训练循环
for epoch in range(EPOCHS):
    model.train()
    total_train_loss = 0.0
    correct_train = 0
    total_train = 0

    # 使用 tqdm 显示训练进度
    train_bar = tqdm(train_loader, desc=f"Train Epoch {epoch+1}/{EPOCHS}")
    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

        train_acc = 100 * correct_train / total_train
        train_bar.set_postfix(loss=loss.item(), acc=f"{train_acc:.2f}%", lr=optimizer.param_groups[0]['lr'])

    # 学习率调整（warmup + cosine）
    if epoch < WARMUP_EPOCHS:
        lr_scale = (epoch + 1) / WARMUP_EPOCHS
    else:
        progress = (epoch - WARMUP_EPOCHS) / (EPOCHS - WARMUP_EPOCHS)
        lr_scale = 0.5 * (1 + math.cos(math.pi * progress))
    current_lr = LR * lr_scale
    for param_group in optimizer.param_groups:
        param_group['lr'] = current_lr

    # 验证阶段
    model.eval()
    correct_val = 0
    total_val = 0
    val_bar = tqdm(val_loader, desc="Validating", leave=False)
    with torch.no_grad():
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_acc = 100 * correct_val / total_val
            val_bar.set_postfix(acc=f"{val_acc:.2f}%")

    # 训练 & 验证结果总结
    epoch_train_loss = total_train_loss / len(train_loader)
    epoch_train_acc = 100 * correct_train / total_train
    epoch_val_acc = 100 * correct_val / total_val

    print(f"[Epoch {epoch+1}/{EPOCHS}] "
          f"Train Loss: {epoch_train_loss:.4f} | "
          f"Train Acc: {epoch_train_acc:.2f}% | "
          f"Val Acc: {epoch_val_acc:.2f}% | "
          f"LR: {current_lr:.6f}")


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/37.1M [00:00<?, ?B/s]

Train Epoch 1/20: 100%|██████████| 803/803 [1:22:04<00:00,  6.13s/it, acc=80.04%, loss=0.345, lr=0.0016]


[Epoch 1/20] Train Loss: 0.4150 | Train Acc: 80.04% | Val Acc: 73.45% | LR: 0.000320


...

Train Epoch 5/20: 100%|██████████| 803/803 [1:18:15<00:00,  5.85s/it, acc=93.70%, loss=0.172, lr=0.00128]


[Epoch 5/20] Train Loss: 0.1566 | Train Acc: 93.70% | Val Acc: 87.61% | LR: 0.001600


Train Epoch 6/20: 100%|██████████| 803/803 [1:19:44<00:00,  5.96s/it, acc=93.10%, loss=0.122, lr=0.0016]


[Epoch 6/20] Train Loss: 0.1592 | Train Acc: 93.10% | Val Acc: 88.73% | LR: 0.001600


Train Epoch 7/20:  51%|█████     | 410/803 [44:03<38:49,  5.93s/it, acc=93.94%, loss=0.353, lr=0.0016]